# Importing Libraries
in this part we will install all the necessary libraries on command prompt and then import the necessary functions from those libraries. 

In [7]:
# importing all the necessary libraries
import pandas as pd

# step 1: preprocessing
from sklearn.impute import SimpleImputer # import some strategic imputer to fill in any missing values using mean
from sklearn.preprocessing import MinMaxScaler # scale all the values to one range to avoid any biasness (this bias is seen in mostly naive bayes and knn etc)
from sklearn.impute import KNNImputer # import some strategic imputer to fill missing values using KNN (finds the nearest neighbour and fills it with that value)

# step 2: data division
from sklearn.model_selection import train_test_split # to divide the code into train/test using a specific percentage or with/without replacement

# step 3: decision trees creation and tuning
from sklearn.tree import DecisionTreeClassifier # to intialize a decision tree maker, its hyper parameters can be defined or set by a grid
from sklearn.model_selection import GridSearchCV # to set different hyper parameters for creating a decision tree

# step 4: displaying accuracy
from sklearn.metrics import roc_auc_score # to display the accuracy of our tree

In [6]:
# use this block to install any libraries not on the system
# !pip install pandas
# !pip install sklearn
# python -m pip install scikit-learn lightgbm xgboost catboost

# Data Loading
data shall be loaded into variables as data sets using pandas and csv readers. they will be checked to see if they are loaded properly and will be loaded as 2 sets: train and test as per given in the kaggle data

In [5]:
# lets load the training data set
train_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\train_set.csv")

In [15]:
train_data.head() # lets also check it by getting the first few rows of the data, there should be x1 - x78 and one target variable Y

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,7,0.134028,93.000000,301.000000,410.010000,0,4.000000,6.0,0.0,5.000000,46.000000,0.0,69.000000,39.000000,35.900000,0.0,96.000000,59.000000,96.000000,59.000000,76.000000,50.000000,110.000000,69.000000,110.000000,69.000000,36.000000,18.000000,100.000000,97.000000,170.000000,101.000000,170.000000,101.000000,36.700000,35.900000,61.000000,59.000000,61.000000,59.000000,61.000000,52.000000,74.000000,71.000000,74.000000,71.000000,27.000000,19.000000,100.000000,99.000000,105.000000,102.000000,105.000000,102.000000,83.000000,83.000000,4.200000,4.200000,0.110000,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,4,0.203574,84.118204,260.708348,337.974019,0,4.000000,6.0,0.0,5.000000,58.437953,0.0,75.029213,54.532835,36.097079,0.0,74.656929,57.591260,74.656929,57.591260,81.715355,69.372284,98.656929,67.875905,98.656929,67.875905,35.788267,16.934331,95.313858,90.343071,137.437953,102.649685,137.437953,102.649685,36.887591,36.009488,72.248189,71.153307,72.248189,71.153307,79.744567,74.839449,96.467165,93.182520,96.467165,93.182520,19.562047,16.934331,93.751811,92.656929,135.248189,133.496378,135.248189,133.496378,139.897874,139.897874,3.718976,3.718976,0.100292,0.033431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,7,0.116667,75.700000,302.000000,105.010000,0,4.000000,6.0,0.0,5.000000,55.000000,0.0,129.000000,42.000000,36.200000,0.0,113.000000,65.000000,113.000000,65.000000,86.000000,55.000000,137.000000,83.000000,137.000000,83.000000,31.000000,7.000000,98.000000,91.000000,163.000000,107.000000,163.000000,107.000000,37.400000,36.200000,113.000000,82.000000,113.000000,82.000000,86.000000,60.000000,137.000000,102.000000,137.000000,102.000000,30.000000,20.000000,97.000000,93.000000,163.000000,126.000000,163.000000,126.000000,109.000000,109.000000,3.800000,3.800000,0.020000,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,7,0.052654,73.956249,121.015914,413.698989,0,3.015914,6.0,0.0,5.000000,50.718754,0.0,141.665798,31.238716,36.608249,0.0,93.127315,49.222801,93.127315,49.222801,94.018521,54.270545,98.366031,66.190973,98.366031,66.190973,31.015914,15.968171,99.015914,92.079572,146.079572,100.015914,146.079572,100.015914,37.050796,36.608249,82.302373,80.968171,82.302373,80.968171,80.193579,79.970778,96.397860,92.111401,96.397860,92.111401,19.143229,18.952257,98.031829,97.984086,149.015914,132.936342,149.015914,132.936342,161.697627,85.254630,4.290451,4.285677,0.108249,0.039363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,5,1.550794,52.311866,169.455275,136.996895,0,4.000000,6.0,0.0,4.308058,108.919416,0.0,51.232234,38.459708,36.461612,0.0,80.066306,45.540292,80.066306,45.540292,103.459708,87.383883,89.682423,58.232234,89.682423,58.232234,31.691942,18.383883,100.000000,97.000000,122.990481,91.530773,122.990481,91.530773,37.715165,36.461612,77.601839,76.677663,77.601839,76.677663,102.075825,101.459708,86.909897,86.601839,86.909897,86.601839,31.691942,30.767766,99.691942,99.075825,116.829313,114.364845,116.829313,114.364845,114.450189,109.213196,3.769194,3.769194,0.164645,0.069242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [12]:
# lets load the test data
test_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\test_set.csv")

In [14]:
test_data.head() # check if the data has been loaded by getting the first 5 rows - there should be x1 - x78 and no target variable Y as this is test data

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78
0,300001,79.000000,17.122318,0,0,1,170.200000,1,700,0,4,0.118056,49.600000,118.000000,602.120000,0,3.000000,4.000000,0.0,4.000000,134.000000,0.0,59.000000,11.000000,36.100000,0.000000,82.000000,63.000000,82.000000,63.000000,106.000000,80.000000,113.000000,89.000000,113.000000,89.000000,16.000000,16.000000,95.0,91.000000,151.000000,123.000000,151.000000,123.000000,37.600000,36.100000,80.000000,80.000000,80.000000,80.000000,104.000000,104.000000,113.000000,113.000000,113.000000,113.000000,16.000000,16.000000,95.0,95.000000,151.000000,151.000000,151.000000,151.000000,138.000000,138.000000,3.300000,3.300000,0.070000,0.030000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,300002,38.000000,43.693579,0,0,1,165.100000,1,814,0,7,0.010417,119.100000,122.000000,703.030000,0,1.000000,1.000000,0.0,1.000000,41.000000,1.0,113.000000,16.000000,36.600000,1.000000,111.000000,47.000000,111.000000,47.000000,79.000000,41.000000,125.000000,60.000000,125.000000,60.000000,31.000000,16.000000,100.0,92.000000,167.000000,99.000000,167.000000,99.000000,37.300000,36.600000,108.000000,63.000000,108.000000,63.000000,46.000000,42.000000,125.000000,84.000000,125.000000,84.000000,16.000000,16.000000,100.0,100.000000,159.000000,131.000000,159.000000,131.000000,352.000000,83.000000,4.600000,3.900000,0.050000,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,300003,36.064225,23.998944,0,0,1,167.086735,1,662,0,7,0.004261,67.592737,122.000000,703.030000,0,2.610444,5.305222,0.0,1.915666,100.116446,0.0,89.602340,12.610444,36.730522,0.694778,104.895558,58.148559,104.895558,58.148559,107.200780,82.811224,116.148559,68.064225,116.148559,68.064225,24.084334,12.915666,100.0,92.220888,150.602340,100.674669,150.602340,100.674669,37.247389,36.669478,104.895558,71.253001,104.895558,71.253001,102.200780,88.200780,116.148559,83.337335,116.148559,83.337335,18.220888,14.136555,100.0,93.610444,150.297118,117.285114,150.297118,117.285114,145.457985,93.875450,3.577911,3.577911,0.006948,0.006948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,300004,61.846764,31.693449,0,3,1,182.355708,2,862,0,7,0.008327,105.981746,167.488094,272.921476,0,3.684676,6.000000,0.0,5.000000,130.738706,0.0,142.513736,35.775030,36.931532,0.000000,97.558913,57.189560,97.558913,57.189560,127.792735,92.414530,142.513736,71.604090,142.513736,71.604090,40.504884,14.162088,100.0,94.261294,153.748473,110.018620,153.748473,110.018620,37.920726,36.931532,78.820207,70.297619,78.820207,70.297619,105.883089,96.567765,95.550061,81.504884,95.550061,81.504884,28.099206,25.099206,100.0,98.000000,144.550061,118.586385,144.550061,118.586385,223.029302,153.875152,4.216209,4.010806,0.062613,0.033153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,300005,71.591991,20.086147,1,0,1,166.704917,2,335,0,6,0.000000,56.072948,302.514572,1257.439179,1,4.000000,6.000000,0.0,5.000000,49.689433,0.0,80.039155,23.533704,35.120400,0.000000,82.087430,60.029143,82.087430,60.029143,75.485428,51.204004,104.514572,80.883426,104.514572,80.883426,19.883426,10.485428,100.0,94.970857,176.397998,131.058287,176.397998,131.058287,37.042714,34.914572,69.806007,64.660289,69.806007,64.660289,60.631146,54.233148,90.145717,85.514572,90.145717,85.514572,17.825139,11.941713,100.0,98.514572,140.835150,133.116574,140.835150,133.116574,153.979978,105.437152,4.139800,4.139800,0.014854,0.004854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Preprocessing
before we start processing this data and using algorithms, we will fix this data first